In [1]:
#importing necessary libraries
from transformers import AutoProcessor, FlavaModel, FlavaImageModel, AutoImageProcessor, FlavaFeatureExtractor
import numpy as np
from torchmultimodal.models.flava.model import flava_model_for_classification

import torchvision
import torch
import torch.nn as nn



/home/IAIS/jraghu/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Checking for GPU and settingt he device as GPU if avaialble
if torch.cuda.is_available():
    print("CUDA is available on this system.")
    device = torch.device("cuda")
else:
    print("CUDA is not available on this system.")
    device = torch.device("cpu")




CUDA is available on this system.


In [3]:
print(torch.cuda.memory_summary(device=device, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

In [4]:
from tqdm import tqdm
from avalanche.benchmarks.generators import nc_benchmark, ni_benchmark
from avalanche.benchmarks.datasets import MNIST

from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop
import torchvision.transforms as transforms



resize_transform = transforms.Resize((224, 224))
grayscale_transform = transforms.Grayscale(num_output_channels=3)
train_transform = Compose([
    resize_transform,
    grayscale_transform,
    ToTensor()
    
])

test_transform = Compose([
    resize_transform,
    grayscale_transform,
    ToTensor()

])


mnist_train = MNIST(
    './data/mnist', train=True, download=True, transform=train_transform
)

mnist_test = MNIST(
    './data/mnist', train=False, download=True, transform=test_transform
)


In [5]:
# Get the first image from the dataset
image, label = mnist_train[0]

# Check the size and shape of the image
image_size = image.size
image_shape = image.shape

print(f"Image Size: {image_size}")
print(f"Image Shape: {image_shape}")

Image Size: <built-in method size of Tensor object at 0x7f962a05b130>
Image Shape: torch.Size([3, 224, 224])


In [6]:
#Classsic class incremental creation from MNIST dataset
# scenario_CIL = nc_benchmark(
#     mnist_train, mnist_test, n_experiences=10, shuffle=True, seed=1234,
#     task_labels=False
# )


#Classsic Domain incremental creation from MNIST dataset
scenario_DIL = ni_benchmark(
    mnist_train, mnist_test, n_experiences=3, shuffle=True, seed=1234,
    balance_experiences=True
)


Freeze the FLAVA ViT model and use only a simple MLP on top of it

In [23]:
#creating a FLAVA custom class which calls the pretrained image model (IM encoder only) and returns the features/embeddings instead of a flava basepooling type

class flava_custom(torch.nn.Module):
    def __init__(self, num_classes, device='cuda'):
        super().__init__()
        self.image_processor = AutoImageProcessor.from_pretrained("facebook/flava-full")
        self.model_custom = FlavaImageModel.from_pretrained("facebook/flava-full").eval().to("cuda")

        self.mlp = nn.Sequential(
                nn.Linear(768, 100),
                nn.ReLU(),
                nn.Linear(100, 50),
                nn.ReLU(),
                nn.Linear(50, 10),
                nn.Sigmoid()
        )

        self.device = device

    def forward(self, x):
        if not isinstance(x, list):
            x = list(x.cuda())
         
        with torch.no_grad():
            inputs = self.image_processor(images=x, return_tensors="pt").to("cuda")
            outputs = self.model_custom(**inputs)
            
            
        
        return self.mlp(outputs.last_hidden_state[:, 0, :])



In [24]:

im_features= flava_custom(num_classes=10) 

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [26]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics, amca_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin, GEMPlugin, GDumbPlugin
from avalanche.training.supervised import Naive, ER_ACE, der, MER
from avalanche.training.templates import SupervisedTemplate
from transformers import AutoImageProcessor, ViTModel


# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    amca_metrics(),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario_DIL.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)


# CREATE THE STRATEGY INSTANCE (EWC)
cl_strategy_DIL = Naive(
    im_features, SGD(im_features.parameters(), lr=0.01, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=100, train_epochs=5, eval_mb_size=25, 
    evaluator=eval_plugin, device=device)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario_DIL.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy_DIL.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy_DIL.eval(scenario_DIL.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-- >> Start of training phase << --
0it [00:00, ?it/s]

100%|██████████| 200/200 [05:15<00:00,  1.58s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2732631.0967
	DiskUsage_MB/train_phase/train_stream/Task000 = 2732631.0967
	Loss_Epoch/train_phase/train_stream/Task000 = 2.3019
	Loss_MB/train_phase/train_stream/Task000 = 2.2994
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0070
	Time_Epoch/train_phase/train_stream/Task000 = 314.8510
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.1131
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.1414
100%|██████████| 200/200 [05:13<00:00,  1.57s/it]
Epoch 1 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2732696.8555
	DiskUsage_MB/train_phase/train_stream/Task000 = 2732696.8555
	Loss_Epoch/train_phase/train_stream/Task000 = 2.3014
	Loss_MB/train_phase/train_stream/Task000 = 2.3029
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0063
	Time_Epoch/train_phase/train_stream/Task000 = 313.6945
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.1124
	

Using the same 2-MLP as the classification head but this time using the complete hidden state with the sequence length too (FLAVA model is still frozen)

In [7]:
import timm
model_res50 = timm.create_model('resnet50', num_classes=10, pretrained=False)

In [7]:
class flava_custom_with_sequence_length(torch.nn.Module):
    def __init__(self, num_classes, device='cuda'):
        super().__init__()
        self.image_processor = AutoImageProcessor.from_pretrained("facebook/flava-full")
        self.model_custom = FlavaImageModel.from_pretrained("facebook/flava-full").eval().to("cuda")

        self.mlp = nn.Sequential(
                nn.Linear(768*197, 100),
                nn.ReLU(),
                nn.Linear(100, 50),
                nn.ReLU(),
                nn.Linear(50, 10),
                nn.Sigmoid()
        )
        self.device = device

    def forward(self, x):
        if not isinstance(x, list):
            x = list(x.cuda())
         
        with torch.no_grad():
            inputs = self.image_processor(images=x, return_tensors="pt").to("cuda")
            outputs = self.model_custom(**inputs)
            
        features = outputs.last_hidden_state.view(outputs.last_hidden_state.size(0), -1)  # Flatten spatial dimensions
        logits = self.mlp(features)    
        
        #print("logits", logits.shape)
        return logits

In [8]:
Cl_model_with_seq = flava_custom_with_sequence_length(num_classes=10)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [9]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics, amca_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin, GEMPlugin, GDumbPlugin
from avalanche.training.supervised import Naive, ER_ACE, der, MER
from avalanche.training.templates import SupervisedTemplate
from transformers import AutoImageProcessor, ViTModel


# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    amca_metrics(),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario_DIL.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger]#, tb_logger]
)


# CREATE THE STRATEGY INSTANCE (EWC)
cl_strategy_DIL = Naive(
    Cl_model_with_seq, SGD(Cl_model_with_seq.parameters(), lr=0.08, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=100, train_epochs=5, eval_mb_size=25, 
    evaluator=eval_plugin, device=device)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario_DIL.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy_DIL.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy_DIL.eval(scenario_DIL.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-- >> Start of training phase << --
100%|██████████| 200/200 [05:23<00:00,  1.62s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2733638.4062
	DiskUsage_MB/train_phase/train_stream/Task000 = 2733638.4062
	Loss_Epoch/train_phase/train_stream/Task000 = 1.7909
	Loss_MB/train_phase/train_stream/Task000 = 1.5655
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0125
	Time_Epoch/train_phase/train_stream/Task000 = 323.4087
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6713
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.8990
100%|██████████| 200/200 [05:21<00:00,  1.61s/it]
Epoch 1 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2733638.4062
	DiskUsage_MB/train_phase/train_stream/Task000 = 2733638.4062
	Loss_Epoch/train_phase/train_stream/Task000 = 1.5351
	Loss_MB/train_phase/train_stream/Task000 = 1.5677
	RunningTime_Epoch/train_phase/train_stream/

Using the same 2-MLP as the classification head but this time using the complete hidden state with the sequence length too (FLAVA model is not frozen anymore)

In [7]:
class flava_custom_with_sequence_length_unfreeze(torch.nn.Module):
    def __init__(self, num_classes, device='cuda'):
        super().__init__()
        self.image_processor = AutoImageProcessor.from_pretrained("facebook/flava-full")
        self.model_custom = FlavaImageModel.from_pretrained("facebook/flava-full").eval().to("cuda")

        self.mlp = nn.Sequential(
                nn.Linear(768*197, 100),
                nn.ReLU(),
                nn.Linear(100, 50),
                nn.ReLU(),
                nn.Linear(50, 10),
                nn.Sigmoid()
        )
        self.device = device

    def forward(self, x):
        if not isinstance(x, list):
            x = list(x.cuda())
         
        #with torch.no_grad():
        inputs = self.image_processor(images=x, return_tensors="pt").to("cuda")
        outputs = self.model_custom(**inputs)
            
        features = outputs.last_hidden_state.view(outputs.last_hidden_state.size(0), -1)  # Flatten spatial dimensions
        logits = self.mlp(features)    
        
        #print("logits", logits.shape)
        return logits

In [8]:
Cl_model_with_seq_unfreeze = flava_custom_with_sequence_length_unfreeze(num_classes=10)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [9]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics, amca_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin, GEMPlugin, GDumbPlugin
from avalanche.training.supervised import Naive, ER_ACE, der, MER
from avalanche.training.templates import SupervisedTemplate
from transformers import AutoImageProcessor, ViTModel


# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    amca_metrics(),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario_DIL.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger]#, tb_logger]
)


# CREATE THE STRATEGY INSTANCE (Naive)
cl_strategy_DIL = Naive(
    Cl_model_with_seq_unfreeze, SGD(Cl_model_with_seq_unfreeze.parameters(), lr=0.05, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=100, train_epochs=5, eval_mb_size=25, 
    evaluator=eval_plugin, device=device)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario_DIL.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy_DIL.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy_DIL.eval(scenario_DIL.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-- >> Start of training phase << --
100%|██████████| 200/200 [07:46<00:00,  2.33s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2733602.6328
	DiskUsage_MB/train_phase/train_stream/Task000 = 2733602.6328
	Loss_Epoch/train_phase/train_stream/Task000 = 2.1115
	Loss_MB/train_phase/train_stream/Task000 = 1.6507
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0194
	Time_Epoch/train_phase/train_stream/Task000 = 466.5134
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3291
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.8182
100%|██████████| 200/200 [07:38<00:00,  2.29s/it]
Epoch 1 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2733602.6328
	DiskUsage_MB/train_phase/train_stream/Task000 = 2733602.6328
	Loss_Epoch/train_phase/train_stream/Task000 = 1.5650
	Loss_MB/train_phase/train_stream/Task000 = 1.5569
	RunningTime_Epoch/train_phase/train_stream/

CL on unfrozen FLAVA model with 2 layered MLP classification head using EWC CL strategy

In [11]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics, amca_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin, GEMPlugin, GDumbPlugin
from avalanche.training.supervised import Naive, ER_ACE, der, MER, EWC
from avalanche.training.templates import SupervisedTemplate
from transformers import AutoImageProcessor, ViTModel


# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    amca_metrics(),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario_DIL.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger]#, tb_logger]
)


# CREATE THE STRATEGY INSTANCE (EWC)
cl_strategy_DIL = EWC(
    Cl_model_with_seq_unfreeze, SGD(Cl_model_with_seq_unfreeze.parameters(), lr=0.09, momentum=0.9),
    CrossEntropyLoss(), ewc_lambda=0.4, train_mb_size=100, train_epochs=5, eval_mb_size=25, 
    evaluator=eval_plugin, device=device)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario_DIL.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy_DIL.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy_DIL.eval(scenario_DIL.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-- >> Start of training phase << --
0it [00:00, ?it/s]

100%|██████████| 200/200 [07:39<00:00,  2.30s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2733622.5361
	DiskUsage_MB/train_phase/train_stream/Task000 = 2733622.5361
	Loss_Epoch/train_phase/train_stream/Task000 = 2.3017
	Loss_MB/train_phase/train_stream/Task000 = 2.2994
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0165
	Time_Epoch/train_phase/train_stream/Task000 = 458.7494
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.1104
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.1111
100%|██████████| 200/200 [07:30<00:00,  2.25s/it]
Epoch 1 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2733622.5361
	DiskUsage_MB/train_phase/train_stream/Task000 = 2733622.5361
	Loss_Epoch/train_phase/train_stream/Task000 = 2.3016
	Loss_MB/train_phase/train_stream/Task000 = 2.2932
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0097
	Time_Epoch/train_phase/train_stream/Task000 = 450.3414
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.1124
	

CL on frozen FLAVA model with 2 layered MLP classification head using EWC CL strategy

In [9]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics, amca_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin, GEMPlugin, GDumbPlugin
from avalanche.training.supervised import Naive, ER_ACE, der, EWC
from avalanche.training.templates import SupervisedTemplate
from transformers import AutoImageProcessor, ViTModel


# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    amca_metrics(),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario_DIL.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger]#, tb_logger]
)


# CREATE THE STRATEGY INSTANCE (EWC)
cl_strategy_DIL = EWC(
    Cl_model_with_seq, SGD(Cl_model_with_seq.parameters(), lr=0.08, momentum=0.9),
    CrossEntropyLoss(), ewc_lambda=0.4, train_mb_size=100, train_epochs=5, eval_mb_size=25, 
    evaluator=eval_plugin, device=device)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario_DIL.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy_DIL.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy_DIL.eval(scenario_DIL.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-- >> Start of training phase << --
100%|██████████| 200/200 [05:20<00:00,  1.60s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2736144.4756
	DiskUsage_MB/train_phase/train_stream/Task000 = 2736144.4756
	Loss_Epoch/train_phase/train_stream/Task000 = 1.7909
	Loss_MB/train_phase/train_stream/Task000 = 1.5655
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0129
	Time_Epoch/train_phase/train_stream/Task000 = 320.1502
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6713
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.8990
100%|██████████| 200/200 [05:19<00:00,  1.60s/it]
Epoch 1 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2736144.4756
	DiskUsage_MB/train_phase/train_stream/Task000 = 2736144.4756
	Loss_Epoch/train_phase/train_stream/Task000 = 1.5351
	Loss_MB/train_phase/train_stream/Task000 = 1.5677
	RunningTime_Epoch/train_phase/train_stream/